In [1]:
!pip install pdfminer3

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [16]:
from pdfminer3.layout import LAParams, LTTextBox
from pdfminer3.pdfpage import PDFPage
from pdfminer3.pdfinterp import PDFResourceManager
from pdfminer3.pdfinterp import PDFPageInterpreter
from pdfminer3.converter import PDFPageAggregator
from pdfminer3.converter import TextConverter
import io
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd
import os
import csv
import re
from pdfminer import high_level
import nltk
from nltk.tokenize import word_tokenize
nltk. download("punkt")

[nltk_data] Downloading package punkt to /home/shreya/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [17]:
#Function to extract Phone Numbers from string using regular expressions
def extract_phone_numbers(string):
    r = re.compile(r'(\d{3}[-\.\s]??\d{3}[-\.\s]??\d{4}|\(\d{3}\)\s*\d{3}[-\.\s]??\d{4}|\d{3}[-\.\s]??\d{4})')
    phone_numbers = r.findall(string)
    #return [re.sub(r'\D', '', number) for number in phone_numbers]
    
    numbers = [re.sub(r'\D', '', number) for number in phone_numbers]
    for item in numbers:
        if len(item) == 10 or len(item) == 14:
            return item

In [18]:
def extract_email_addresses(string):
    r = re.compile(r'[\w\.-]+@[\w\.-]+')
#     r = re.compile('^(\w|\.|\_|\-)+[@](\w|\_|\-|\.)+[.]\w{2,3}$')  
    return r.findall(string)

In [19]:
with open('techskill.csv', newline='') as csvfile:
        data = list(csv.reader(csvfile))
        
def extract_skills(extracted_text):
    skills = []
   
    a_list = nltk. word_tokenize(extracted_text.lower()) #Split text into list of words.
    for item in data:
        for sub_item in item:
            if sub_item in a_list:
                skills.append(sub_item)
    return skills

In [20]:
resource_manager = PDFResourceManager()
fake_file_handle = io.StringIO()
converter = TextConverter(resource_manager, fake_file_handle, laparams=LAParams())
page_interpreter = PDFPageInterpreter(resource_manager, converter)

with open("Job description.pdf", 'rb') as fh:

    for page in PDFPage.get_pages(fh,
                                  caching=True,
                                  check_extractable=True):
        page_interpreter.process_page(page)

    text2 = fake_file_handle.getvalue()

# close open handles
converter.close()
fake_file_handle.close()

#print(text2)
#type(text2)

In [34]:
def read_resume(extracted_text):
    resource_manager = PDFResourceManager()
    fake_file_handle = io.StringIO()
    converter = TextConverter(resource_manager, fake_file_handle, laparams=LAParams())
    page_interpreter = PDFPageInterpreter(resource_manager, converter)
    


    text = extracted_text

    # close open handles
    converter.close()
    fake_file_handle.close()

    #print(text)
    #type(text)
    #creating a list
    list1 = [text,text2]
    cv = CountVectorizer()
    count_matrix = cv.fit_transform(list1)
    #print("\nSimilarity scores")
    #print(cosine_similarity(count_matrix))
    matchPercentage = cosine_similarity(count_matrix)[0][1] * 100
    matchPercentage = round(matchPercentage,2) #round it to 2 decimal places
    #print("The resume matches about "+str(matchPercentage)+ "% of job description")
    return matchPercentage

In [35]:

cols = ["name","phone_number","email","skills","matchPercentage"]
rows = []
d = 'Resumes'
def select_files_in_folder(d, ext):
    for file in os.listdir(d):
        if file.endswith('.%s' % ext):
            yield os.path.join(d, file)

for file in select_files_in_folder(d, 'pdf'):
    pages = [0,1] # just the first page
    extracted_text = high_level.extract_text(file, "", pages)
    #name = extract_names(extracted_text)
    name = file[8:(len(file))-4]
    phone_number = extract_phone_numbers(extracted_text)
    email = extract_email_addresses(extracted_text)
    skills = extract_skills(extracted_text)
    percentage = read_resume(extracted_text)
    rows.append({'name': name,
                     'phone_number':phone_number,
                        'email':email[0],
                         'skills':skills,
                            'matchPercentage':percentage})
out_df = pd.DataFrame(rows, columns = cols)

In [36]:
print(out_df)

                                                 name phone_number  \
0                                 Abhijay Kumar Singh   7760380991   
1                                          T S Kushal   9449938598   
2   Divya_Madhav_Kamath_Resume_April42018 - Divya ...   9591721577   
3                        Resume_july - Rakesh gourani   9164260924   
4                          praveen-resume - Praveen G   9743175960   
5                                        Rutwik Nayak   9964442953   
6                                        Shwet Sharma   9482026516   
7                Apoorva_R_Resume - Apoorva Bharadwaj   7204441749   
8                            Nikhil Prabhakar Kottary   9187489821   
9                                            Aarthi T   8762068961   
10          Amogha_Subramanya - Amogha Subramanya D A   8553511856   
11                                    Ashritha Shetty   9191081432   
12                               SRM_ML - Sourabh R M   9880125575   
13                  